In [85]:
%%javascript
require.config({
   'shim': {'bootstrap': {'deps' :['jquery']},
            'igv': {'deps' :['jquery', 'jquery-ui', 'bootstrap']}
           },
    
   paths: {cytoscape  :  'http://cytoscape.github.io/cytoscape.js/api/cytoscape.js-latest/cytoscape.min.js',
           //cytoscape  : 'http://localhost:8099/js/cytoscape-2.7.10',
          'jquery'    :   'http://code.jquery.com/jquery-1.12.4.min',
          'jquery-ui' :   'http://code.jquery.com/ui/1.12.1/jquery-ui.min',
          'bootstrap' :   'http://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min',
          'igv'       :   'http://igv.org/web/release/1.0.6/igv-1.0.6'
          }
   })

<IPython.core.display.Javascript object>

In [86]:
# ensure libraries are available
import requests
#assert(requests.get('http://localhost:8099/js/cytoscape-2.7.10.js').status_code == 200)
#assert(requests.get('https://igv.org/web/beta/igv-beta.min.js').status_code == 200)

In [87]:
import ipywidgets as widgets
import time
from IPython.display import display, HTML
from traitlets import Int, Unicode, observe

In [88]:
class TabsWidget(widgets.DOMWidget):
    
    _view_name = Unicode('TabsView').tag(sync=True)
    _view_module = Unicode('tabsDemo').tag(sync=True)
    frameHeight = Int(300).tag(sync=True)

    def setHeight(self, height):
       print("setHeight(%d) "% height)
       self.frameHeight = height
    

In [89]:
display(HTML(data="""
<style>
    div#notebook-container    { width: 97%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [90]:
%%javascript
"use strict";

require.undef('tabsDemo')

define('tabsDemo', ["jupyter-js-widgets", "cytoscape", "jquery", "igv"], 
       function(widgets, cytoscape, $, igv) {
    
    var TabsView = widgets.DOMWidgetView.extend({

        initialize: function() {
           this.options = {}
           console.log("constructing TabsView");
           this.frameHeight = "800px";
           },

        resizeHandler: function(){
           console.log("TabsView resizeHandler")            
           },

       createCytoscapeDiv: function(){
          var cyDiv = $("<div id='cyDiv' style='border:1px solid red; height:200px;'></div>");
          cyDiv.cytoscape({
            ready: function(){
               x = this;
               console.log("cyJS ready");
               },
            elements: {
              nodes: [
                {data: {id: 'a', name: 'Node A', type: 'big' }},
                {data: {id: 'b', name: 'Node B', type: 'little'}},
                ]}
            });
          return(cyDiv);
          }, // createCytoscapeDiv
        
        createMasterTabsDiv: function(){
           var masterTabsDiv = $("<div id='masterTabsDiv' style='border:1px solid gray; height: 400px; width: 97%'></div>");
           var list = $("<ul/>"); // .appendTo(masterTabsDiv);
           list.append("<li><a href='#tab-1'>cytoscape</a></li>");
           list.append("<li><a href='#tab-2'>igv</a></li>");
           list.append("<li><a href='#tab-3'>three</a></li>");
           masterTabsDiv.append(list);
           var tab1 = $("<div id='tab-1'>contents 1</div>");
           tab1.append("<div id='cyDiv' style='border:1px solid red; height:200px;'>cyDiv coming soon</div>"); 
           //var tab1 = this.createCytoscapeDiv();
           var tab2 = $("<div id='tab-2'></div>");
           tab2.append("<div id='igvDiv' style='border:1px solid blue; height:500px;'></div>");
           var tab3 = $("<div id='tab-3'>contents 3</div>");
           masterTabsDiv.append(tab1);
           masterTabsDiv.append(tab2);
           masterTabsDiv.append(tab3);
           return(masterTabsDiv);
           },
         

        render: function() { 
            console.log("entering render");
            this.masterTabsDiv = this.createMasterTabsDiv();
            this.$el.append(this.masterTabsDiv);
            this.listenTo(this.model, 'change:frameHeight', this.frameDimensionsChanged, this);
            //var browser = None;
            var igvOptions = {
                 palette: ["#00A0B0", "#6A4A3C", "#CC333F", "#EB6841"],
                 locus: "7:55,085,725 - 55,276,031",
                 reference: {id: "hg19",
                   fastaURL: "http://igv.broadinstitute.org/genomes/seq/1kg_v37/human_g1k_v37_decoy.fasta",
                   cytobandURL: "http://igv.broadinstitute.org/genomes/seq/b37/b37_cytoband.txt"
                   },
                 trackDefaults: {
                    bam: {coverageThreshold: 0.2,
                          coverageQualityWeight: true
                         }
                    },
               tracks: [
                {name: "Genes",
                    url: "http://igv.broadinstitute.org/annotations/hg19/genes/gencode.v18.collapsed.bed",
                    index: "http://igv.broadinstitute.org/annotations/hg19/genes/gencode.v18.collapsed.bed.idx",
                    displayMode: "EXPANDED"
                    }
                 ]
               }; // igvOptions
 
            setTimeout(function(){
               console.log("about to call tabs()");
               $("#masterTabsDiv").tabs();
               $("#cyDiv").cytoscape({elements:{nodes:[{data:{id:'a'}}], edges:[{data:{source:'a', target:'a'}}]}});
               window.browser = igv.createBrowser($("#igvDiv"), igvOptions);
               }, 0);
            },

           
        frameDimensionsChanged: function(){
           console.log("frameDimensionsChanged");
           var oldHeight = $("#mainDiv").height()
           var oldWidth = $("#mainDiv").width()
           var newHeight = this.model.get("frameHeight");
           var msg = "<center>tabs demo, height: " + oldHeight + " -> " + newHeight + "</center>";
           $("#mainDiv").html(msg);
           $("#masterTabsDiv").height(newHeight);
        }, 
        

    });
    return {
        TabsView: TabsView
    };
});

<IPython.core.display.Javascript object>

In [91]:
app = TabsWidget()
display(app)

In [77]:
app.setHeight(800)


setHeight(800) 
